# Import Dependencies

In [ ]:
%%capture
!pip install ensemble-boxes

In [ ]:
from ensemble_boxes import *
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as backend
import collections
from tensorflow.keras import *
import tensorflow_addons as tfa
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np
import string

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from glob import glob
import random
import math
from tqdm.notebook import tqdm
import os


import matplotlib.pyplot as plt
import cv2
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
    #keras.mixed_precision.set_global_policy("mixed_bfloat16")
except:
    TPU = None
    strategy = tf.distribute.get_strategy() 
N_REPLICAS = 8
tf.config.optimizer.set_jit(True)
bs = 2 # TPU's compile faster with smaller BS.
BATCH_SIZE = bs * N_REPLICAS
TARGET_DTYPE =  tf.float32

# Load in the Model Fully.

Generate Anchors

In [ ]:
def get_anchors(image_size):
    all_anchors = []
    grid_size = np.array([8, 16, 32, 64, 128], np.float32)
    anchor_sizes = grid_size * 4.
    anchor_ratios = np.array([0.5, 1, 2], np.float32)
    anchor_scales = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)], np.float32)
    for i in range(5):
        # anchors (1, 9, 4)
        anchors_list = np.zeros((len(anchor_ratios) * len(anchor_scales), 4))
        anchors_list[:, 2] = np.tile(anchor_sizes[i] * anchor_scales, 3) / np.sqrt(np.repeat(anchor_ratios, len(anchor_scales)))
        anchors_list[:, 3] = np.tile(anchor_sizes[i] * anchor_scales, 3) * np.sqrt(np.repeat(anchor_ratios, len(anchor_scales)))
        anchors_list[:, 0::2] -= anchors_list[:, 2:3] * 0.5
        anchors_list[:, 1::2] -= anchors_list[:, 3:4] * 0.5
        anchors_list = np.expand_dims(anchors_list, axis=0)
        
        # shift (K, 1, 4)
        shift_pts = np.arange(grid_size[i] // 2, image_size, grid_size[i], dtype=np.float32)  # [4, 12, 30, ...]
        shift_list = np.zeros((len(shift_pts) ** 2, 4)) # (K, 1, 4)
        shift_list[:,0] = shift_list[:,2] = np.tile(shift_pts, len(shift_pts)) # x1 x2   np.tile (0, 1, 2) => (0, 1, 2, 0, 1, 2, 0, 1, 2)
        shift_list[:,1] = shift_list[:,3] = np.repeat(shift_pts, len(shift_pts)) # y1 y2  np.repeat (0, 1, 2) => (0, 0, 0, 1, 1, 1, 2, 2, 2)
        shift_list = np.expand_dims(shift_list, axis=1)
        
        # merge (K, 9, 4) = > (K * 9, 4)
        all_anchors.append(np.reshape(anchors_list + shift_list, (-1, 4)))
    all_anchors = np.concatenate(all_anchors, axis=0)
    return tf.cast(all_anchors, tf.float32)

Helper FN's.

In [ ]:
def compute_area(bbox):
    # Computes Areas of a Bbox
    ones = bbox[:, :2] # (N, 2)
    twos = bbox[:, 2:4] # (N, 2)
    diff = tf.maximum(twos - ones, 0.0) # (N, 2)
    return diff[:, 0] * diff[:, 1] # (N,)
def filter_bboxes(box):
    # Filters out Bboxes if their area is < 0
    indices_for_object = tf.where(tf.math.reduce_all([box[:,4] >= 0, 
                                                          box[:, 2] >= box[:, 0], 
                                                          box[:, 3] >= box[:, 1]], axis=0))
    box = tf.cond(tf.shape(indices_for_object)[0] == 0, 
                    lambda: tf.reshape([0., 0., -1., -1., 0.], (-1, 1, box.shape[-1])),
                    lambda: tf.reshape(tf.gather(box, indices_for_object), (-1, 1, box.shape[-1]))) # (?, 1, 5)
    return box
def compute_centers(bbox):
    return 0.5 * (bbox[:, :2] + bbox[:, 2:4])
def compute_iou(bbox, anchors):
    # Computes IOU Between Bbox and Anchors
    # Bbox: tensor(N, min(4)) - As long as the first 4 values are (x1, y1, x2, y2)
    # Anchors: (x1, y1, x2, y2)
    
    # Expand bbox and anchor for broadcasting
    area_bbox = compute_area(bbox) # (N, )
    area_anchor = compute_area(anchors) # (M,)
    
    area_bbox = tf.expand_dims(area_bbox, axis = 0) # (1, N)
    area_anchor = tf.expand_dims(area_anchor, axis = 1) # (M, 1)
    
    
    bbox = tf.expand_dims(bbox, axis = 0) # (1, N, 5)
    anchors = tf.expand_dims(anchors, axis = 1) # (M, 1, 5)
    top_left_bboxes = tf.maximum(bbox[:, :, :2], anchors[:, :, :2]) # (M, N, 2)
    bottom_right_bbox = tf.minimum(bbox[:, :, 2:4], anchors[:, :, 2:4]) # (M, N, 2)
    
    
    
    differences = bottom_right_bbox - top_left_bboxes# (M, N, 2)
    differences = tf.maximum(differences, 0.0)
    
    inter = tf.maximum(differences[:, :, 0] * differences[:, :, 1], 0.0) # (M, N)
    union = area_bbox + area_anchor - inter # (M, N)

    eps = 1e-6
    return (inter + eps) / (union + eps) # (M, N)

Data Module + Loading in Data 

In [ ]:
class DataModule:
    data_dir = 'test-gwd'
    if TPU:
        data_dir = KaggleDatasets().get_gcs_path(data_dir)
        data_dir += '/*.tfrec'
        glob_function = lambda x: tf.io.gfile.glob(x)
    else:
        data_dir = f"../input/{data_dir}/*.tfrec"
        glob_function = lambda x: glob(x)
    # grab all files 
    all_files = glob_function(data_dir)
 
    NUM_FILES = sum(1 for _ in tf.data.TFRecordDataset(all_files))
    
    REQUIRED_DATASET_PAD = BATCH_SIZE-NUM_FILES%BATCH_SIZE 
    IMG_SHAPE = (1024, 1024, 3)
    
    MAX_BBOXES = 200 # Maximum number of Wheat Heads Possible
    NUM_TTA = 4
    # ANCHOR BOXES:
    ANCHORS = get_anchors(IMG_SHAPE[0])

In [ ]:
def load_image(example):
    feature_dict = {
        'image': tf.io.FixedLenFeature(shape=[], dtype=tf.string, default_value=''),
        'image_id': tf.io.FixedLenFeature(shape = [], dtype = tf.string, default_value="")
    }
    features = tf.io.parse_single_example(example, features=feature_dict)   
    image = features['image']
    image_id = features['image_id']
    
    
    # Load image
    image = tf.io.decode_jpeg(image, channels = 3)
    # Cast to Desired Dtype
    image = tf.cast(image, tf.float32) / 255.0
    # Reshape the image 
    image = tf.reshape(image,  DataModule.IMG_SHAPE) # (1024, 1024, 3)
    
    # TTA 4
    image0 = tf.identity(image)
    image1 = tf.image.flip_left_right(tf.identity(image))
    image2 = tf.image.flip_up_down(tf.identity(image))
    image3 = tf.image.flip_up_down(tf.image.flip_left_right(tf.identity(image)))
    
    return image0, image1, image2, image3, image_id

In [ ]:
def load_test_data():
    '''
    Load in the test data
    '''
    dataset = tf.data.TFRecordDataset(
        DataModule.all_files,
        num_parallel_reads = AUTO
    )
    options = tf.data.Options()
    options.experimental_deterministic = False
    
    dataset = dataset.with_options(options)
    dataset = dataset.map(lambda x: load_image(x), num_parallel_calls = AUTO, deterministic = False)
    
    # CREATE PADDING DATASET
    if DataModule.REQUIRED_DATASET_PAD != 0:
        extra_padding = DataModule.REQUIRED_DATASET_PAD
        pad_dataset = tf.data.Dataset.from_tensor_slices((
            tf.zeros((extra_padding, *DataModule.IMG_SHAPE), dtype=TARGET_DTYPE),  
            tf.zeros((extra_padding, *DataModule.IMG_SHAPE), dtype = TARGET_DTYPE),
            tf.zeros((extra_padding, *DataModule.IMG_SHAPE), dtype = TARGET_DTYPE),
            tf.zeros((extra_padding, *DataModule.IMG_SHAPE), dtype = TARGET_DTYPE),# Fake Images
            tf.constant(["000000000000",]*extra_padding, dtype=tf.string))   # Fake IDs
        )
        dataset = dataset.concatenate(pad_dataset)
        
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset
    

In [ ]:
dataset = load_test_data()

# load in the model

In [ ]:
def increment_index(indices, index):
  begin = indices[:index]
  end = indices[index + 1:]
  middle = tf.expand_dims(indices[index] + 1, axis = 0)
  indices = tf.concat([begin, middle, end], axis = 0)
  return indices
def replace_index_double_bbox(bboxes, first_index, second_index, new_value):
  # Double Index
  first_index = tf.squeeze(first_index)
  second_index = tf.squeeze(second_index)
  
  begin = bboxes[:first_index]
  end = bboxes[first_index + 1:]

  middle = bboxes[first_index]
  middle = tf.expand_dims(replace_index_bbox(middle, second_index, new_value), axis = 0)
  new_bboxes = tf.concat([begin, middle, end], axis = 0)
  return new_bboxes


In [ ]:
def display_images(images, bboxes):
    values = tf.where((tf.greater_equal(bboxes[0][:, 4], 1.0)))
    idx = tf.squeeze(tf.where((tf.greater_equal(bboxes[0][:, 4], 1.0))))
    bounding = tf.gather(bboxes[0], idx)
    anchors = tf.gather(DataModule.ANCHORS, idx)
    images = images.numpy()
    bounding = decode_bounding_box(bounding, anchors).numpy()
    for idx in range(len(bounding)):
        bbox = bounding[idx, :] # (5)
        x1, y1, x2, y2 = bbox[:4]
        coord1 = (x1, y1)
        coord2 = (x2, y2)


        cv2.rectangle(images, (int(x1), int(y1)), (int(x2), int(y2)), 220, 3)
    plt.imshow(images)
    plt.show()
    

def decode_bounding_box(bboxes, anchors, image_size = 1024):
    # Decodes the Predictions of the Bboxes(N,5) - Only X1,Y1, X2, Y2
    width = anchors[:, 2] - anchors[:, 0]
    height = anchors[:, 3] - anchors[:, 1]
    # Obj Score should be kept the same for NMS.
    
    x1 = tf.expand_dims(bboxes[:, 0] + anchors[:, 0], axis = 1)
    y1 = tf.expand_dims(bboxes[:, 1] + anchors[:, 1], axis = 1)
    w =  tf.expand_dims(tf.math.exp(bboxes[:, 2]) * width, axis = 1)
    h = tf.expand_dims(tf.math.exp(bboxes[:, 3]) * height, axis = 1)
    obj =bboxes[:, 4:]
    
    
    x2 = x1 + w
    y2 = y1 + h
    
    new_bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 1)
    return new_bboxes
    

In [ ]:
def EfficientNetBN(n, input_tensor=None, input_shape=None, **kwargs):
    CONV_KERNEL_INITIALIZER = {
        'class_name': 'VarianceScaling',
        'config': {
            'scale': 2.0,
            'mode': 'fan_out',
            # EfficientNet actually uses an untruncated normal distribution for
            # initializing conv layers, but keras.initializers.VarianceScaling use
            # a truncated distribution.
            # We decided against a custom initializer for better serializability.
            'distribution': 'normal'
        }
    }

    def get_swish():
        def swish(x):
            return x * tf.math.sigmoid(x)
        return swish


    def get_dropout():
        class FixedDropout(layers.Dropout):
            def _get_noise_shape(self, inputs):
                if self.noise_shape is None:
                    return self.noise_shape
                symbolic_shape = tf.shape(inputs)
                noise_shape = [symbolic_shape[axis] if (shape is None) else shape for axis, shape in enumerate(self.noise_shape)]
                return tuple(noise_shape)
        return FixedDropout


    def round_filters(filters, width_coefficient, depth_divisor):
        filters *= width_coefficient
        new_filters = int(filters + depth_divisor / 2) // depth_divisor * depth_divisor
        new_filters = max(depth_divisor, new_filters)
        if new_filters < 0.9 * filters:
            new_filters += depth_divisor
        return int(new_filters)


    def round_repeats(repeats, depth_coefficient):
        return int(math.ceil(depth_coefficient * repeats))


    def mb_conv_block(inputs, block_args, activation, drop_rate=None, prefix='', freeze_bn=False):
        has_se = (block_args.se_ratio is not None) and (0 < block_args.se_ratio <= 1)
        bn_axis = 3 

        Dropout = get_dropout()

        filters = block_args.input_filters * block_args.expand_ratio
        if block_args.expand_ratio != 1:
            x = layers.Conv2D(filters, 1, padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER, name=prefix + 'expand_conv')(inputs)
            x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'expand_bn')(x)
            x = layers.Activation(activation, name=prefix + 'expand_activation')(x)
        else:
            x = inputs

        # Depthwise Convolution
        x = layers.DepthwiseConv2D(block_args.kernel_size, strides=block_args.strides, padding='same', use_bias=False, depthwise_initializer=CONV_KERNEL_INITIALIZER, name=prefix + 'dwconv')(x)
        x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'bn')(x)
        x = layers.Activation(activation, name=prefix + 'activation')(x)

        # Squeeze and Excitation phase
        if has_se:
            num_reduced_filters = max(1, int(block_args.input_filters * block_args.se_ratio))
            se_tensor = layers.GlobalAveragePooling2D(name=prefix + 'se_squeeze')(x)

            target_shape = (1, 1, filters) if backend.image_data_format() == 'channels_last' else (filters, 1, 1)
            se_tensor = layers.Reshape(target_shape, name=prefix + 'se_reshape')(se_tensor)
            se_tensor = layers.Conv2D(num_reduced_filters, 1, activation=activation, padding='same', use_bias=True, kernel_initializer=CONV_KERNEL_INITIALIZER, name=prefix + 'se_reduce')(se_tensor)
            se_tensor = layers.Conv2D(filters, 1, activation='sigmoid', padding='same', use_bias=True, kernel_initializer=CONV_KERNEL_INITIALIZER, name=prefix + 'se_expand')(se_tensor)
            if backend.backend() == 'theano':
                pattern = ([True, True, True, False] if (backend.image_data_format() == 'channels_last') else [True, False, True, True])
                se_tensor = layers.Lambda(lambda x: backend.pattern_broadcast(x, pattern), name=prefix + 'se_broadcast')(se_tensor)
            x = layers.multiply([x, se_tensor], name=prefix + 'se_excite')

        # Output phase
        x = layers.Conv2D(block_args.output_filters, 1, padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER, name=prefix + 'project_conv')(x)
        x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'project_bn')(x)
        if block_args.id_skip and all(s == 1 for s in block_args.strides) and block_args.input_filters == block_args.output_filters:
            if drop_rate and (drop_rate > 0):
                x = Dropout(drop_rate, noise_shape=(None, 1, 1, 1), name=prefix + 'drop')(x)
            x = layers.add([x, inputs], name=prefix + 'add')
        return x


    def EfficientNet(width_coefficient, depth_coefficient, drop_connect_rate=0.2, depth_divisor=8, input_tensor=None, input_shape=None, freeze_bn=False, **kwargs):
        BlockArgs = collections.namedtuple('BlockArgs', [
            'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
            'expand_ratio', 'id_skip', 'strides', 'se_ratio'
        ])
        blocks_args = [
            BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, strides=[1, 1], se_ratio=0.25),
            BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24, expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
            BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40, expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
            BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, output_filters=80, expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
            BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, output_filters=112, expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25),
            BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, output_filters=192, expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
            BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, output_filters=320, expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25)
        ]
        
        features = []

        img_input = layers.Input(shape=input_shape) if (input_tensor is None) else (input_tensor)

        bn_axis = 3 
        activation = get_swish(**kwargs)

        # Build stem
        x = img_input

        x = layers.Conv2D(round_filters(32, width_coefficient, depth_divisor), 3, strides=(2, 2), padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER, name='stem_conv')(x)
        x = layers.BatchNormalization(axis=bn_axis, name='stem_bn')(x)
        x = layers.Activation(activation, name='stem_activation')(x)

        # Build blocks
        num_blocks_total = sum(block_args.num_repeat for block_args in blocks_args)
        block_num = 0
        for idx, block_args in enumerate(blocks_args):
            assert block_args.num_repeat > 0
            # Update block input and output filters based on depth multiplier.
            block_args = block_args._replace(
                input_filters=round_filters(block_args.input_filters, width_coefficient, depth_divisor),
                output_filters=round_filters(block_args.output_filters, width_coefficient, depth_divisor),
                num_repeat=round_repeats(block_args.num_repeat, depth_coefficient))

            # The first block needs to take care of stride and filter size increase.
            drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
            x = mb_conv_block(x, block_args, activation=activation, drop_rate=drop_rate, prefix='block{}a_'.format(idx + 1), freeze_bn=freeze_bn)
            block_num += 1
            if block_args.num_repeat > 1:
                # pylint: disable=protected-access
                block_args = block_args._replace(
                    input_filters=block_args.output_filters, strides=[1, 1])
                # pylint: enable=protected-access
                for bidx in range(block_args.num_repeat - 1):
                    drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
                    block_prefix = 'block{}{}_'.format(idx + 1, string.ascii_lowercase[bidx + 1])
                    x = mb_conv_block(x, block_args, activation=activation, drop_rate=drop_rate, prefix=block_prefix, freeze_bn=freeze_bn)
                    block_num += 1
            if idx < len(blocks_args) - 1 and blocks_args[idx + 1].strides[0] == 2:
                features.append(x)
            elif idx == len(blocks_args) - 1:
                features.append(x)
        return features

    
    parms = [
        { "width_coefficient" : 1.0, "depth_coefficient" : 1.0, "default_resolution" : 224},
        { "width_coefficient" : 1.0, "depth_coefficient" : 1.1, "default_resolution" : 240},
        { "width_coefficient" : 1.1, "depth_coefficient" : 1.2, "default_resolution" : 260},
        { "width_coefficient" : 1.2, "depth_coefficient" : 1.4, "default_resolution" : 300},
        { "width_coefficient" : 1.4, "depth_coefficient" : 1.8, "default_resolution" : 380},
        { "width_coefficient" : 1.6, "depth_coefficient" : 2.2, "default_resolution" : 456},
        { "width_coefficient" : 1.8, "depth_coefficient" : 2.6, "default_resolution" : 528},
        { "width_coefficient" : 2.0, "depth_coefficient" : 3.1, "default_resolution" : 600},
    ][n]
    return EfficientNet(parms['width_coefficient'], parms['depth_coefficient'], input_tensor=input_tensor, input_shape=input_shape, **kwargs)

#print(EfficientNetBN(7, input_shape=(600, 600, 3)))

In [ ]:
MOMENTUM = 0.99
EPSILON = 1e-3

class wBiFPNAdd(layers.Layer):
    def __init__(self, epsilon=1e-4, **kwargs):
        super(wBiFPNAdd, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        num_in = len(input_shape)
        self.w = self.add_weight(name=self.name, shape=(num_in,), initializer=initializers.constant(1 / num_in), trainable=True, dtype= tf.float32)

    def call(self, inputs, **kwargs):
        w = tf.cast(activations.relu(self.w), dtype = inputs[0].dtype)
        x = tf.reduce_sum([w[i] * inputs[i] for i in range(len(inputs))], axis=0)
        x = x / (tf.reduce_sum(w) + self.epsilon)
        return x

    def compute_output_shape(self, input_shape):
        return input_shape[0]

    def get_config(self):
        config = super(wBiFPNAdd, self).get_config()
        config.update({ 'epsilon': self.epsilon })
        return config
    
    
def SeparableConvBlock(num_channels, kernel_size, strides, name, freeze_bn=False):
    f1 = layers.SeparableConv2D(num_channels, kernel_size=kernel_size, strides=strides, padding='same',
                                use_bias=True, name=f'{name}/conv')
    f2 = layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON, name=f'{name}/bn')
    return lambda *args, **kwargs: f2(f1(*args, **kwargs))


def build_wBiFPN(features, num_channels, id, freeze_bn=False):
    if id == 0:
        _, _, C3, C4, C5 = features
        
        # 第一次BIFPN需要 下采样 与 降通道 获得 p3_in p4_in p5_in p6_in p7_in
        #-----------------------------下采样 与 降通道----------------------------#
        P3_in = C3
        
        P3_in = layers.Conv2D(num_channels, kernel_size=1, padding='same',
                              name=f'fpn_cells/cell_{id}/fnode3/resample_0_0_8/conv2d')(P3_in)
        P3_in = layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON,
                                          name=f'fpn_cells/cell_{id}/fnode3/resample_0_0_8/bn')(P3_in)

        P4_in = C4
        P4_in_1 = layers.Conv2D(num_channels, kernel_size=1, padding='same',
                                name=f'fpn_cells/cell_{id}/fnode2/resample_0_1_7/conv2d')(P4_in)
        P4_in_1 = layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON,
                                            name=f'fpn_cells/cell_{id}/fnode2/resample_0_1_7/bn')(P4_in_1)
        P4_in_2 = layers.Conv2D(num_channels, kernel_size=1, padding='same',
                                name=f'fpn_cells/cell_{id}/fnode4/resample_0_1_9/conv2d')(P4_in)
        P4_in_2 = layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON,
                                            name=f'fpn_cells/cell_{id}/fnode4/resample_0_1_9/bn')(P4_in_2)

        P5_in = C5
        P5_in_1 = layers.Conv2D(num_channels, kernel_size=1, padding='same',
                                name=f'fpn_cells/cell_{id}/fnode1/resample_0_2_6/conv2d')(P5_in)
        P5_in_1 = layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON,
                                            name=f'fpn_cells/cell_{id}/fnode1/resample_0_2_6/bn')(P5_in_1)
        P5_in_2 = layers.Conv2D(num_channels, kernel_size=1, padding='same',
                                name=f'fpn_cells/cell_{id}/fnode5/resample_0_2_10/conv2d')(P5_in)
        P5_in_2 = layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON,
                                            name=f'fpn_cells/cell_{id}/fnode5/resample_0_2_10/bn')(P5_in_2)

        P6_in = layers.Conv2D(num_channels, kernel_size=1, padding='same', name='resample_p6/conv2d')(C5)
        P6_in = layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON, name='resample_p6/bn')(P6_in)
        P6_in = layers.MaxPooling2D(pool_size=3, strides=2, padding='same', name='resample_p6/maxpool')(P6_in)

        P7_in = layers.MaxPooling2D(pool_size=3, strides=2, padding='same', name='resample_p7/maxpool')(P6_in)
        #-------------------------------------------------------------------------#
        #--------------------------构建BIFPN的上下采样循环-------------------------#
        P7_U = layers.UpSampling2D(dtype = tf.float16)(P7_in)
        P6_td = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode0/add')([P6_in, P7_U])
        P6_td = layers.Activation(lambda x: tf.nn.swish(x))(P6_td)
        P6_td = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                   name=f'fpn_cells/cell_{id}/fnode0/op_after_combine5')(P6_td)
        
        P6_U = layers.UpSampling2D()(P6_td)
        P5_td = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode1/add')([P5_in_1, P6_U])
        P5_td = layers.Activation(lambda x: tf.nn.swish(x))(P5_td)
        P5_td = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                   name=f'fpn_cells/cell_{id}/fnode1/op_after_combine6')(P5_td)

        P5_U = layers.UpSampling2D()(P5_td)
        P4_td = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode2/add')([P4_in_1, P5_U])
        P4_td = layers.Activation(lambda x: tf.nn.swish(x))(P4_td)
        P4_td = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                   name=f'fpn_cells/cell_{id}/fnode2/op_after_combine7')(P4_td)

        P4_U = layers.UpSampling2D()(P4_td)
        P3_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode3/add')([P3_in, P4_U])
        P3_out = layers.Activation(lambda x: tf.nn.swish(x))(P3_out)
        P3_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode3/op_after_combine8')(P3_out)

        P3_D = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(P3_out)
        P4_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode4/add')([P4_in_2, P4_td, P3_D])
        P4_out = layers.Activation(lambda x: tf.nn.swish(x))(P4_out)
        P4_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode4/op_after_combine9')(P4_out)

        P4_D = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(P4_out)
        P5_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode5/add')([P5_in_2, P5_td, P4_D])
        P5_out = layers.Activation(lambda x: tf.nn.swish(x))(P5_out)
        P5_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode5/op_after_combine10')(P5_out)

        P5_D = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(P5_out)
        P6_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode6/add')([P6_in, P6_td, P5_D])
        P6_out = layers.Activation(lambda x: tf.nn.swish(x))(P6_out)
        P6_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode6/op_after_combine11')(P6_out)

        P6_D = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(P6_out)
        P7_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode7/add')([P7_in, P6_D])
        P7_out = layers.Activation(lambda x: tf.nn.swish(x))(P7_out)
        P7_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode7/op_after_combine12')(P7_out)

    else:
        P3_in, P4_in, P5_in, P6_in, P7_in = features
        # Change the Dtypes to float16
        
        
        P7_U = layers.UpSampling2D()(P7_in)
        P6_td = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode0/add')([P6_in, P7_U])
        P6_td = layers.Activation(lambda x: tf.nn.swish(x))(P6_td)
        P6_td = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                   name=f'fpn_cells/cell_{id}/fnode0/op_after_combine5')(P6_td)

        P6_U = layers.UpSampling2D()(P6_td)
        P5_td = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode1/add')([P5_in, P6_U])
        P5_td = layers.Activation(lambda x: tf.nn.swish(x))(P5_td)
        P5_td = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                   name=f'fpn_cells/cell_{id}/fnode1/op_after_combine6')(P5_td)

        P5_U = layers.UpSampling2D()(P5_td)
        P4_td = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode2/add')([P4_in, P5_U])
        P4_td = layers.Activation(lambda x: tf.nn.swish(x))(P4_td)
        P4_td = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                   name=f'fpn_cells/cell_{id}/fnode2/op_after_combine7')(P4_td)

        P4_U = layers.UpSampling2D()(P4_td)
        
        P3_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode3/add')([P3_in, P4_U])
        P3_out = layers.Activation(lambda x: tf.nn.swish(x))(P3_out)
        P3_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode3/op_after_combine8')(P3_out)

        P3_D = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(P3_out)
        P4_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode4/add')([P4_in, P4_td, P3_D])
        P4_out = layers.Activation(lambda x: tf.nn.swish(x))(P4_out)
        P4_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode4/op_after_combine9')(P4_out)

        P4_D = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(P4_out)
        P5_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode5/add')([P5_in, P5_td, P4_D])
        P5_out = layers.Activation(lambda x: tf.nn.swish(x))(P5_out)
        P5_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode5/op_after_combine10')(P5_out)

        P5_D = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(P5_out)
        P6_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode6/add')([P6_in, P6_td, P5_D])
        P6_out = layers.Activation(lambda x: tf.nn.swish(x))(P6_out)
        P6_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode6/op_after_combine11')(P6_out)

        P6_D = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(P6_out)
        P7_out = wBiFPNAdd(name=f'fpn_cells/cell_{id}/fnode7/add')([P7_in, P6_D])
        P7_out = layers.Activation(lambda x: tf.nn.swish(x))(P7_out)
        P7_out = SeparableConvBlock(num_channels=num_channels, kernel_size=3, strides=1,
                                    name=f'fpn_cells/cell_{id}/fnode7/op_after_combine12')(P7_out)
    return [P3_out, P4_out, P5_out, P6_out, P7_out]


class PriorProbability(initializers.Initializer):
    """ Apply a prior probability to the weights.
    """
    def __init__(self, probability=0.01):
        self.probability = probability

    def get_config(self):
        return { 'probability': self.probability }

    def __call__(self, shape, dtype=None):
        # set bias to -log((1 - p)/p) for foreground
        result = np.ones(shape) * -math.log((1 - self.probability) / self.probability)

        return result

    
class BoxNet(layers.Layer):
    def __init__(self, width, depth, num_anchors=9, freeze_bn=False, name='box_net', **kwargs):
        super().__init__()
        
        self.width = width
        self.depth = depth
        self.num_anchors = num_anchors
        options = {
            'kernel_size': 3,
            'strides': 1,
            'padding': 'same',
            'bias_initializer': 'zeros',
            'depthwise_initializer': initializers.VarianceScaling(),
            'pointwise_initializer': initializers.VarianceScaling(),
        }

        self.convs = [layers.SeparableConv2D(filters=width, name=f'{name}/box-{i}', **options) for i in range(depth)]
        self.head = layers.SeparableConv2D(filters=num_anchors * 4, name=f'{name}/box-predict', **options)

        self.bns = [
            [layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON, name=f'{name}/box-{i}-bn-{j}') for j in
             range(3, 8)]
            for i in range(depth)]

        self.relu = layers.Lambda(lambda x: tf.nn.swish(x))
        self.reshape = layers.Reshape((-1, 4))

    def call(self, inputs):
        feature, level = inputs
        for i in range(self.depth):
            feature = self.convs[i](feature)
            feature = self.bns[i][level](feature)
            feature = self.relu(feature)
        outputs = self.head(feature)
        outputs = self.reshape(outputs)
        return outputs


class ClassNet(layers.Layer):
    def __init__(self, width, depth, num_classes=20, num_anchors=9, freeze_bn=False, name='class_net', **kwargs):
        super().__init__()
        
        self.width = width
        self.depth = depth
        self.num_classes = num_classes
        self.num_anchors = num_anchors
        options = {
            'kernel_size': 3,
            'strides': 1,
            'padding': 'same',
            'depthwise_initializer': initializers.VarianceScaling(),
            'pointwise_initializer': initializers.VarianceScaling(),
        }

        self.convs = [layers.SeparableConv2D(filters=width, bias_initializer='zeros', name=f'{name}/class-{i}', **options) for i in range(depth)]
        self.head = layers.SeparableConv2D(filters=num_classes * num_anchors, bias_initializer=PriorProbability(probability=0.01), name=f'{name}/class-predict', **options)
        self.bns = [[layers.BatchNormalization(momentum=MOMENTUM, epsilon=EPSILON, name=f'{name}/class-{i}-bn-{j}') for j in range(3, 8)] for i in range(depth)]
        self.relu = layers.Lambda(lambda x: tf.nn.swish(x))
        self.reshape = layers.Reshape((-1, num_classes))
        self.activation = layers.Activation('sigmoid')

    def call(self, inputs):
        feature, level = inputs
        for i in range(self.depth):
            feature = self.convs[i](feature)
            feature = self.bns[i][level](feature)
            feature = self.relu(feature)
        outputs = self.head(feature)
        outputs = self.reshape(outputs)
        outputs = self.activation(outputs)
        return outputs

def Efficientdet(phi, num_classes=20, num_anchors=9, freeze_bn=True):
    assert phi in range(8)
    fpn_num_filters = [64, 88, 112, 160, 224, 288, 384,384]
    fpn_cell_repeats = [3, 4, 5, 6, 7, 7, 8, 8]
    box_class_repeats = [3, 3, 3, 4, 4, 4, 5, 5]
    image_sizes = [512, 640, 768, 896, 1024, 1280, 1408, 1536]
    
    image_input = layers.Input((image_sizes[phi], image_sizes[phi], 3), name='input', dtype = TARGET_DTYPE)
    features = EfficientNetBN(phi, input_tensor=image_input, freeze_bn=freeze_bn)
 
    fpn_features = features
    if phi < 6:
        for i in range(fpn_cell_repeats[phi]):
            fpn_features = build_wBiFPN(fpn_features, fpn_num_filters[phi], i, freeze_bn=freeze_bn)
    else:        
        for i in range(fpn_cell_repeats[phi]):
            fpn_features = build_BiFPN(fpn_features, fpn_num_filters[phi], i, freeze_bn=freeze_bn)

    box_net = BoxNet(fpn_num_filters[phi], box_class_repeats[phi], num_anchors=num_anchors, freeze_bn=freeze_bn, name='box_net')
    class_net = ClassNet(fpn_num_filters[phi], box_class_repeats[phi], num_classes=num_classes, num_anchors=num_anchors, freeze_bn=freeze_bn, name='class_net')
    
    classification = [class_net([feature, i]) for i, feature in enumerate(fpn_features)]
    classification = layers.Concatenate(axis=1, name='classification')(classification)
    regression = [box_net([feature, i]) for i, feature in enumerate(fpn_features)]
    regression = layers.Concatenate(axis=1, name='regression')(regression)

    model = models.Model(inputs=[image_input], outputs=[regression, classification], name='efficientdet')

    return model

In [ ]:
class ModelConfig:
  num_classes = 1
  phi = 4
  model_path = [
      '../input/gwd-efficientdetd4/best_model_0.h5',
      
  ]
  num_models = len(model_path)

In [ ]:
def load_model():
    all_models = []
    for model_path in ModelConfig.model_path:
        model = Efficientdet(4, num_classes=1)
        model.load_weights(model_path)
        all_models.append(model)
    return all_models

# NMS CODE 

In [ ]:
def replace_index_bbox(bboxes, idx, new_bbox):
  N, C = bboxes.shape
  # Bboxes: Tensor(N, 6)
  # Idx: Tensor(1)
  # New_BBox: Tensor(6)
  begin = bboxes[:idx]
  end = bboxes[idx + 1:]
  middle = tf.expand_dims(new_bbox, axis = 0)

  new_bbox = tf.concat([begin, middle, end], axis = 0)
  new_bbox = tf.reshape(new_bbox, (-1, C))
  return new_bbox
def replace_index_scores(scores, idx, new_score):
  N = scores.shape[0]

  begin = scores[:idx]
  end = scores[idx + 1:]
  middle = tf.expand_dims(new_score, axis = 0)

  new_score = tf.concat([begin, middle, end], axis = 0)
  new_score = tf.reshape(new_score, (-1, ))
  return new_score 

In [ ]:
def replace_index(array, new_val, index):
    left = array[:index]
    middle = new_val
    right = array[index + 1:]
    new = tf.concat([left, middle, right], axis = 0)
    return tf.reshape(new, (len(array), ))

In [ ]:
def soft_nms_float(dets, labels, Nt, sigma, thresh, method):
    """
    Based on: https://github.com/DocF/Soft-NMS/blob/master/soft_nms.py
    It's different from original soft-NMS because we have float coordinates on range [0; 1]
    :param dets:   boxes format [x1, y1, x2, y2]
    :param sc:     scores for boxes
    :param Nt:     required iou 
    :param sigma:  
    :param thresh: 
    :param method: 1 - linear soft-NMS, 2 - gaussian soft-NMS, 3 - standard NMS
    :return: index of boxes to keep
    """

    # indexes concatenate boxes with the last column
    N = dets.shape[0]
    indexes = tf.cast(tf.expand_dims(tf.range(N), 1), dtype = dets.dtype)
    dets = tf.concat([dets, indexes], axis=1) # (N, 6)

    # the order of boxes coordinate is [y1, x1, y2, x2]
    y1 = dets[:, 1] # (N, )
    x1 = dets[:, 0] # (N, )
    y2 = dets[:, 3] # (N, )
    x2 = dets[:, 2] # (N, )

    scores = dets[:, 4] # (N, )
    areas = (x2 - x1) * (y2 - y1) # (N, )

    for i in range(N - 1):
        # intermediate parameters for later parameters exchange
        tBD = tf.identity(dets[i, :])
        tscore = tf.identity(scores[i])
        tarea = tf.identity(areas[i])
        pos = i + 1
        if i != N - 1:
            maxscore = tf.reduce_max(scores[pos:], axis=0)
            maxpos = tf.argmax(scores[pos:], axis=0)
        else:
            maxscore = scores[-1]
            maxpos = 0
        if tscore < maxscore:
            dets = replace_index_bbox(dets, i, dets[maxpos + pos, :])
            dets = replace_index_bbox(dets, maxpos + pos, tBD)
            tBD = dets[i, :]

            scores = replace_index_scores(scores, i, scores[maxpos + pos])
            scores = replace_index_scores(scores, maxpos + pos, tscore)
            tscore = scores[i]

            areas = replace_index_scores(areas, i, areas[maxpos + pos])
            areas = replace_index_scores(areas, maxpos + pos, tarea)
            tarea = areas[i]

        # IoU calculate
        xx1 = tf.maximum(dets[i, 1], dets[pos:, 1])
        yy1 = tf.maximum(dets[i, 0], dets[pos:, 0])
        xx2 = tf.minimum(dets[i, 3], dets[pos:, 3])
        yy2 = tf.minimum(dets[i, 2], dets[pos:, 2])

        w = tf.maximum(0.0, xx2 - xx1)
        h = tf.maximum(0.0, yy2 - yy1)
        inter = w * h
        ovr = inter / (areas[i] + tf.gather(areas, pos) - inter)
        # Three methods: 1.linear 2.gaussian 3.original NMS
        if method == 1:  # linear
            weight = tf.ones_like(ovr)
            boolean_mask = ovr > Nt 
            N = weight.shape[0]
            for idx in range(N):
              if boolean_mask[idx]:
                weight = replace_index_scores(weight, idx, weight[idx] - ovr[idx])
            
        elif method == 2:  # gaussian
            weight = tf.exp(-(ovr * ovr) / sigma)
        else:  # original NMS
            weight = tf.ones_like(ovr)
            boolean_mask = ovr > Nt
            N = boolean_mask.shape[0]
            for idx in range(N):
              weight = replace_index_scores(weight, idx, tf.constant(0.0, dtype = weight.dtype))

        length_left = len(scores) - pos
        for idx in range(length_left):
          new_idx = idx + pos
          scores = replace_index_scores(scores, new_idx, weight[idx] * scores[new_idx])

    # select the boxes and keep the corresponding indexes
  
    inds = dets[:, 5][scores > thresh]
    # Replace Scores 
    x1 = tf.expand_dims(dets[:, 0], axis = 1)
    y1 = tf.expand_dims(dets[:, 1], axis = 1)
    x2 = tf.expand_dims(dets[:, 2], axis = 1)
    y2 = tf.expand_dims(dets[:, 3], axis = 1)
    scores = tf.expand_dims(scores, axis = 1)

    bboxes = tf.concat([x1, y1, x2, y2, scores], axis = 1)
    # select out good bounding boxes 
    bboxes = tf.gather(bboxes, tf.cast(inds, tf.int64))
    labels = tf.gather(labels, tf.cast(inds, tf.int64))
    
    labels = tf.squeeze(labels)
    return bboxes, labels


def nms_float(dets, labels, thresh):
    """
    # It's different from original nms because we have float coordinates on range [0; 1]
    :param dets: numpy array of boxes with shape: (N, 5). Order: x1, y1, x2, y2, score. All variables in range [0; 1]
    :param thresh: IoU value for boxes
    :return: index of boxes to keep
    """
    MAX_NUM = DataModule.MAX_BBOXES
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]
    
    areas = (x2 - x1) * (y2 - y1)
    order = tf.argsort(scores)[::-1]
    

    keep = tf.ones((MAX_NUM, ), dtype = tf.int32) * tf.constant(-1, dtype = tf.int32)
    cur_idx = 0
    while len(order) != 0:
        i = order[0]

        new_val = tf.reshape(tf.cast(i, dtype = keep.dtype), (1, ) )
        keep = replace_index(keep, new_val, cur_idx)
        cur_idx += 1
        if order.shape[0] == 1:
            break
        i = tf.reshape(i, ())
        xx1 = tf.maximum(x1[i], tf.gather(x1, order[1:]))
        yy1 = tf.maximum(y1[i], tf.gather(y1, order[1:]))
        xx2 = tf.minimum(x2[i], tf.gather(x2, order[1:]))
        yy2 = tf.minimum(y2[i], tf.gather(y2, order[1:]))

        w = tf.maximum(0.0, xx2 - xx1)
        h = tf.maximum(0.0, yy2 - yy1)
        inter = w * h
        ovr = inter / (areas[i] + tf.gather(areas, order[1:]) - inter)
        inds = ovr <= thresh
        order = tf.boolean_mask(order[1:], inds)
    keep = keep[:cur_idx]
    dets = tf.gather(dets, keep)
    labels = tf.gather(labels, keep)
    labels = tf.squeeze(labels)
    
    # PAD THE BOUNDING BOXES And LABELS
    num_pad = MAX_NUM - cur_idx;
    pad_dets = tf.ones((num_pad, 5), dtype = dets.dtype) * tf.constant(-1.0, dtype = dets.dtype)
    pad_labels = tf.ones((num_pad,), dtype = dets.dtype) * tf.constant(-1.0, dtype = dets.dtype)
    
    dets = tf.concat([dets, pad_dets], axis = 0)
    labels = tf.concat([labels, pad_labels], axis = 0)
    
    dets = tf.reshape(dets, (MAX_NUM, 5))
    labels = tf.reshape(labels, (MAX_NUM, ))
    return dets, labels


def nms_method(boxes, labels, method=3, iou_thr=0.5, sigma=0.5, thresh=0.001, weights=None):
    """
    :param boxes: list of boxes predictions from each model, each box is 4 numbers. 
    It has 3 dimensions (models_number, model_preds, 4)
    Order of boxes: x1, y1, x2, y2. We expect float normalized coordinates [0; 1] 
    :param scores: list of scores for each model 
    :param labels: list of labels for each model
    :param method: 1 - linear soft-NMS, 2 - gaussian soft-NMS, 3 - standard NMS
    :param iou_thr: IoU value for boxes to be a match 
    :param sigma: Sigma value for SoftNMS
    :param thresh: threshold for boxes to keep (important for SoftNMS)
    :param weights: list of weights for each model. Default: None, which means weight == 1 for each model
    :return: boxes: boxes coordinates (Order of boxes: x1, y1, x2, y2). 
    :return: scores: confidence scores
    :return: labels: boxes labels
    """

    # Run NMS independently for each label

    if method != 3:
        bboxes, labels = soft_nms_float(boxes, labels, Nt=iou_thr, sigma=sigma, thresh=thresh, method=method)
    else:
        # Use faster function
        bboxes, labels = nms_float(boxes, labels, thresh=iou_thr)


    return bboxes, labels

def nms(boxes, labels, iou_thr=0.5, weights=None):
    """
    Short call for standard NMS 
    
    :param boxes: 
    :param scores: 
    :param labels: 
    :param iou_thr: 
    :param weights: 
    :return: 
    """
    return nms_method(boxes, labels, method=3, iou_thr=iou_thr, weights=weights)


def soft_nms(boxes, labels, method=2, iou_thr=0.5, sigma=0.5, thresh=0.001, weights=None):
    """
    Short call for Soft-NMS
     
    :param boxes: 
    :param scores: 
    :param labels: 
    :param method: 
    :param iou_thr: 
    :param sigma: 
    :param thresh: 
    :param weights: 
    :return: 
    """
    return nms_method(boxes, labels, method=method, iou_thr=iou_thr, sigma=sigma, thresh=thresh, weights=weights)

In [ ]:
def increment_index(indices, index):
  begin = indices[:index]
  end = indices[index + 1:]
  middle = tf.expand_dims(indices[index] + 1, axis = 0)
  indices = tf.concat([begin, middle, end], axis = 0)
  return indices
def replace_index_double_bbox(bboxes, first_index, second_index, new_value):
  # Double Index
  first_index = tf.squeeze(first_index)
  second_index = tf.squeeze(second_index)
  
  begin = bboxes[:first_index]
  end = bboxes[first_index + 1:]

  middle = bboxes[first_index]
  middle = tf.expand_dims(replace_index_bbox(middle, second_index, new_value), axis = 0)
  new_bboxes = tf.concat([begin, middle, end], axis = 0)
  return new_bboxes


In [ ]:
def compute_weighted_box(bboxes, i):
  # Computes an AVG Weighted Box
  # Boxes: Tensor(NUM_BOXES, NUM_BOXES, 5)
  num_bboxes = len(bboxes)
  to_avg_bboxes = bboxes[i] # (NUM_BOXES, 5)
  # Average These Bounding Boxes According to Confi Q
  Y1 = tf.zeros((1, ), dtype = bboxes.dtype)
  X2 = tf.zeros((1, ), dtype = bboxes.dtype)
  Y2 = tf.zeros((1, ), dtype = bboxes.dtype)
  CONF = tf.zeros((1, ), dtype = bboxes.dtype)
 
  for j in range(num_bboxes):
    bbox = to_avg_bboxes[j] # (5)
    
    conf = bbox[4]
    x1 = bbox[0] * conf
    y1 = bbox[1] * conf
    x2 = bbox[2] * conf
    y2 = bbox[3] * conf
    X1 = X1 + x1
    Y1 = Y1 + y1
    X2 = X2 + x2
    Y2 = Y2 + y2
    CONF = CONF + conf

  X1 = X1 / CONF
  Y1 = Y1 / CONF
  X2 = X2 / CONF
  Y2 = Y2 / CONF
  CONF = CONF / tf.cast(num_bboxes, dtype = CONF.dtype)
  
  LBL = tf.ones_like(CONF) * bboxes[0, 0, 5]
  
  new_bbox = tf.concat([X1, Y1, X2, Y2, CONF, LBL], axis = 0) # (5, )
  return new_bbox

# Inference Loop

In [ ]:
def pad_final_bounding_boxes(bounding_boxes):
    # PADS THE NUMBER OF BOUNDING BOXES TO MAX_NUM 
    # Also cuts away anything more than N values
    # SHAPE: Tensor(N, 5)
    MAX_DET = DataModule.MAX_BBOXES
    bounding_boxes =  bounding_boxes[:MAX_DET]
    
    N = bounding_boxes.shape[0]
    num_pad = MAX_DET - N

    pad_bboxes = tf.ones((num_pad, 5), dtype = bounding_boxes.dtype) * tf.constant(-1.0, dtype = bounding_boxes.dtype)
    
    padded_variant = tf.concat([bounding_boxes, pad_bboxes], axis = 0) # (MAX_DET, 5)
    return padded_variant 
def replace_bounding_box_index(bounding_boxes, new_bbox, index):
    '''
    bounding_boxes: Tensor(B, MAX_NUM_BBOXES, 5)
    new_bbox: Tensor(M, 5) 
    '''
    MAX_DET = DataModule.MAX_BBOXES
    # Pad Bounding Boxes
    
    padded_new_bboxes = tf.expand_dims(new_bbox, axis = 0) # (1, 200, 5) 
    # Grab the Values to the left
    left_bboxes = bounding_boxes[:index] # (index, 200, 5)
    # Grab Right Values 
    right_bboxes = bounding_boxes[index + 1:]
    # Reshape
    padded_new_bboxes = tf.reshape(padded_new_bboxes, (-1, MAX_DET, 5))
    left_bboxes = tf.reshape(left_bboxes, (-1, MAX_DET, 5))
    right_bboxes = tf.reshape(right_bboxes, (-1, MAX_DET, 5))
    
    # Concat the all
    new_bounding_boxes = tf.concat([left_bboxes, padded_new_bboxes, right_bboxes], axis = 0) # (B, 200, 5)
    return new_bounding_boxes 
    

In [ ]:
def decode_bounding_boxes(bboxes, anchors):
    # Boxes: Tensor(B, 196196, 4)
    # Anchors: Tensor(B, 196196, 4)
    B = bboxes.shape[0]
    anchors = tf.expand_dims(anchors, axis = 0)
    anchors = tf.repeat(anchors, B, axis = 0) 
    width = anchors[:, :, 2] - anchors[:, :, 0]
    height = anchors[:, :, 3] - anchors[:, :,  1]
    # Obj Score should be kept the same for NMS.
    
    x1 = tf.expand_dims(bboxes[:, :, 0] + anchors[:,:,  0], axis = 2)
    y1 = tf.expand_dims(bboxes[:, :, 1] + anchors[:, :, 1], axis = 2)
    w =  tf.expand_dims(tf.math.exp(bboxes[:, :, 2]) * width, axis = 2)
    h = tf.expand_dims(tf.math.exp(bboxes[:, :, 3]) * height,  axis = 2)
    
    x2 = x1 + w
    y2 = y1 + h
    
    new_bounding_boxes = tf.concat([x1, y1, x2, y2], axis = 2)
    return new_bounding_boxes

In [ ]:
def replace_number_of_bounding_boxes(bounding_boxes, start_idx, end_idx, new_bboxes):
    # Bounding_boxes: Tensor(B, N, 4) 
    # new_bboxes: Tensor(B, N, 4)
    B, N, _ = bounding_boxes.shape
    starting = bounding_boxes[:, :start_idx, :]
    ending = bounding_boxes[:,  end_idx:, :] 
    middle = new_bboxes
    
    bboxes = tf.concat([starting, middle, ending], axis = 1) # (B, N, 4)
    bboxes = tf.reshape(bboxes, (B, N, 5))
    return bboxes

In [ ]:
def inference_code(model, images, threshold = 0.5):
    training = False 
    
    bounding_boxes, classification = model(images, training = training)
    
    # Bounding Boxes: Tensor(B, 196196, 4)
    # Classification: Tensor(B, 196196, 1) 
    # DECODE THE BOUNDING BOXES 
  
    bounding_boxes = decode_bounding_boxes(bounding_boxes, DataModule.ANCHORS)
    B = bounding_boxes.shape[0]
    
    max_dets = DataModule.MAX_BBOXES
    final_dets = tf.ones((B, max_dets, 5), dtype = bounding_boxes.dtype) * tf.constant(-1.0, dtype = bounding_boxes.dtype)
    
    for b in range(B):
        single_batch_bbox = bounding_boxes[b] # (N, 4) 
        single_batch_classes = classification[b] # (N, 1)
      
        # Filter out the Bad Bounding Boxes
        keep = single_batch_classes >= threshold
        keep = tf.reshape(keep, (-1, ))
        kept_bounding_boxes = tf.boolean_mask(single_batch_bbox, keep) # (N, 4)
       
        N = kept_bounding_boxes.shape[0]
        if N == 0:

            # No Detections whatsoever
            continue; # Just Skip, it's padded with -1's.
        kept_classification = tf.expand_dims(tf.boolean_mask(single_batch_classes, keep), axis = 1) # (N, 1)
        # All Bounding Boxes are 1(It's binary classifcation )
        kept_bounding_boxes = tf.reshape(kept_bounding_boxes, (-1, 4))
        kept_classification = tf.reshape(kept_classification, (-1, 1))
        kept_bounding_boxes = tf.concat([kept_bounding_boxes, kept_classification], axis = 1) # (N, 5)
        
        # Non Maximum Suppression on the Bounding Boxes Left 
        detections, labels = nms(kept_bounding_boxes, tf.ones_like(kept_bounding_boxes, dtype = kept_bounding_boxes.dtype)[:, 0], iou_thr = 0.3)
        # They are all label 1, so just add the new values into the final predictions
        final_dets = replace_bounding_box_index(final_dets, detections, b)
    return final_dets
def ensemble_model_pred(images):
    B = images.shape[0]
    NUM_MODELS = ModelConfig.num_models
    MAX_BBOXES = DataModule.MAX_BBOXES
    all_preds = tf.ones((B, MAX_BBOXES * NUM_MODELS, 5), dtype = images.dtype) * tf.constant(-1.0, dtype = images.dtype)
    
    start_idx = 0
    for model in ALL_MODELS:
        end_idx = start_idx + MAX_BBOXES
        det_boxes = inference_code(model, images) # (B, N, 5) 
        all_preds = replace_number_of_bounding_boxes(all_preds, start_idx, end_idx, det_boxes)
        start_idx += MAX_BBOXES
    return all_preds # (B, 4000, 5)

In [ ]:
def display_images(images, bboxes):
    images = images.numpy()
    bounding = bboxes.numpy()
    for idx in range(len(bounding)):
        bbox = bounding[idx, :] # (5)
        x1, y1, x2, y2 = bbox[:4]
        coord1 = (x1, y1)
        coord2 = (x2, y2)
        print(coord1)
        print(coord2)
        cv2.rectangle(images, (int(x1), int(y1)), (int(x2), int(y2)), 220, 3)
    plt.imshow(images)
    plt.show()

In [ ]:
def lr_flip(bboxes):
    # Bbox: (B, N, 4)
    B, N, _ = bboxes.shape
    IMG_CENTER = DataModule.IMG_SHAPE[1] // 2

    x1 = tf.expand_dims(bboxes[:, :, 0], 2)
    y1 = tf.expand_dims(bboxes[:, :, 1], 2)
    x2 = tf.expand_dims(bboxes[:, :, 2], 2)
    y2 = tf.expand_dims(bboxes[:, :, 3], 2)
    obj = bboxes[:, :, 4:]
    # (B, N, 1) 


    x1 = x1 + 2 * (IMG_CENTER - x1)
    x2 = x2 + 2 * (IMG_CENTER - x2) # (B, N, 1) 

    box_w = tf.abs(x2 - x1) # (B, N, 1)

    x1 = x1 - box_w
    x2 = x2 + box_w
    
    boolean_mask = box_w == tf.constant(0.0, dtype = box_w.dtype) # (B, N, 1)
    boolean_mask = tf.constant(1.0, dtype = box_w.dtype) - tf.cast(boolean_mask, dtype = box_w.dtype) # (B, N, 1)
    
    x1 = x1 * boolean_mask
    x2 = x2 * boolean_mask 
    
    bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 2)
    bboxes = tf.reshape(bboxes, (B, N, 5))
    return bboxes
  
def ud_flip(bboxes):
    # Bbox: (B, N, 4)
    B, N, _ = bboxes.shape
    IMG_CENTER = DataModule.IMG_SHAPE[1] // 2

    x1 = tf.expand_dims(bboxes[:, :, 0], 2)
    y1 = tf.expand_dims(bboxes[:, :, 1], 2)
    x2 = tf.expand_dims(bboxes[:, :, 2], 2)
    y2 = tf.expand_dims(bboxes[:, :, 3], 2)
    obj = bboxes[:, :, 4:]
    # (B, N, 1) 


    y1 = y1 + 2 * (IMG_CENTER - y1)
    y2 = y2 + 2 * (IMG_CENTER - y2) # (B, N, 1) 

    box_h = tf.abs(y2 - y1) # (B, N, 1)

    y1 = y1 - box_h
    y2 = y2 + box_h
    
    boolean_mask = box_h == tf.constant(0.0, dtype = box_h.dtype) # (B, N, 1)
    boolean_mask = tf.constant(1.0, dtype = box_h.dtype) - tf.cast(boolean_mask, dtype = box_h.dtype) # (B, N, 1)
    
    y1 = y1 * boolean_mask
    y2 = y2 * boolean_mask 
    
    bboxes = tf.concat([x1, y1, x2, y2, obj], axis = 2)
    bboxes = tf.reshape(bboxes, (B, N, 5))
    return bboxes
  

In [ ]:
@tf.function
def test_dist_step(image0, image1, image2, image3, image_id):
    # TTA 4: LR FLIP, UD Flip, Normal, LR -> UD Flip.
    
    
    bbox0 = ensemble_model_pred(image0)
    
    bbox1 = ensemble_model_pred(image1) # (B, NUM_TTA * MAX_NUM, 5) 
    bbox1 = lr_flip(bbox1)
    
    bbox2 = ensemble_model_pred(image2)
    bbox2 = ud_flip(bbox2)
    
    bbox3 = ensemble_model_pred(image3)
    bbox3 = ud_flip(lr_flip(bbox3))
    
    bboxes = tf.concat([bbox0, bbox1, bbox2, bbox3], axis = 1) 
    bboxes = strategy.gather(bboxes, axis = 0)
    image_id = strategy.gather(image_id, axis = 0)
    return bboxes, image_id

In [ ]:
def single_process(bounding_boxes, image_id):
    bbox = bounding_boxes.numpy() # (MAX_NUM * NUM_TTA *  NUM_MODELS,  4)
    img_id = image_id.numpy().decode()
    predString = ''
  
    # Map Bbox to Img_id
    bbox = np.reshape(bbox, ((ModelConfig.num_models, DataModule.MAX_BBOXES * DataModule.NUM_TTA, 5)))
    all_bounding_boxes = []
    all_scores = []
    all_labels = [] 
    for model_box in bbox:
        # model_box: Tensor(MAX_BBOXES * NUM_TTA, 5) 
        model_box = np.reshape(model_box, (DataModule.NUM_TTA, DataModule.MAX_BBOXES, 5))
        all_model_boxes = []
        all_model_scores = []
        all_model_labels = []
        for tta_box in model_box:
            all_tta_boxes = []
            for box in tta_box:
                if box[0] == -1 and box[1] == -1 and box[2] == -1 and box[3] == -1:
                    # Padding
                    continue
                if box[4] == -1:
                    continue
                x1, y1, x2, y2, obj = box.tolist()
                
                all_tta_boxes.append([x1, y1, x2, y2, obj])
            all_tta_boxes = np.array(all_tta_boxes)
            all_model_boxes.append(all_tta_boxes[:, :4] / DataModule.IMG_SHAPE[1]) # Normalize the Bounding Boxes
            all_model_scores.append(all_tta_boxes[:, 4])
            all_model_labels.append(np.zeros_like(all_tta_boxes[:, 4]))
        # WBF on TTA Boxes 
        all_model_boxes, all_model_scores, all_model_labels = weighted_boxes_fusion(all_model_boxes, all_model_scores, all_model_labels)
        # all_model_boxes: tensor(N, 4) 
        # all_model_scores: tensor(N, )
        # all_model_labels: tensor(N, ) 
        all_bounding_boxes.append(all_model_boxes)
        all_scores.append(all_model_scores)
        all_labels.append(all_model_labels)
    # Weighted box Fusion on Models 
    all_bounding_boxes, all_scores, all_labels = weighted_boxes_fusion(all_bounding_boxes, all_scores, all_labels)
    # all_bounding_boxes: Tensor(N, 4)
    # all_scores: Tensor(N, ) 
    # all_Labels: Tensor(N, ) 

    bbox = all_bounding_boxes * DataModule.IMG_SHAPE[0]
    new_bboxes = []

    for box in bbox:

        x1, y1, x2, y2 = box.tolist()

        x1 = round(x1)
        y1 = round(y1)
        x2 = round(x2)
        y2 = round(y2)

        if x1 < 0:
            x1 = 0
        elif x1 > 1024:
            x1 = 1024

        if y1 < 0:
            y1 = 0
        elif y1 > 1024:
            y1 = 1024

        if x2 < 0:
            x2 = 0
        elif x2 > 1024:
            x2 = 1024

        if y2 < 0:
            y2 = 0
        elif y2 > 1024:
            y2 = 1024

        min_val_y = min(y1, y2)
        max_val_y = max(y1, y2)

        min_val_x = min(x1, x2)
        max_val_x = max(x1, x2)

        x1 = min_val_x
        x2 = max_val_x

        y1 = min_val_y
        y2 = max_val_y
        new_bboxes.append([x1, y1, x2, y2])
    return np.array(new_bboxes)



In [ ]:
ALL_MODELS = load_model()

# 2 Hours to get a TPU runtime
- 7 Minutes/100

In [ ]:
# INFERENCE CODE!
test_ds = load_test_data()
MAX_NUM_BBOXES = DataModule.MAX_BBOXES
predicted_bounding_boxes = tf.ones((0, MAX_NUM_BBOXES * DataModule.NUM_TTA * ModelConfig.num_models, 5), dtype = tf.float32)
predicted_image_ids = tf.zeros((0, ), dtype = tf.string)
count = 0
cur_thresh = 0
for image0, image1, image2, image3, image_id in test_ds:
    bounding_boxes, image_id = test_dist_step(image0, image1, image2, image3, image_id) # (N, 4)
    predicted_bounding_boxes = tf.concat([predicted_bounding_boxes, bounding_boxes,], axis = 0)
    predicted_image_ids  = tf.concat([predicted_image_ids, image_id], axis = 0)
    count += BATCH_SIZE
    
    if count > cur_thresh:
        print(count, len(bounding_boxes[bounding_boxes[:, :, -1] != -1.0])) # Sanity Check every 100 turns.
     
        cur_thresh += 100
        

In [ ]:
def post_process(bounding_boxes, image_id):
  
    path = '../input/aicrowdwheatdata/submission.csv'
    df = pd.read_csv(path) 
    orig_df = pd.read_csv(path)
    mapping = {} 
    # Read in the File and Map Image_Name to Domain 
    for row in df.iterrows():
        row = row[1]
        image_name = row.image_name
        domain = row.domain
        mapping[image_name] = domain
    # PROCESS IN THE BOUNDING BOXES
    mapping_GT = {}
    N = bounding_boxes.shape[0]
    for i in range(N):
        bbox = bounding_boxes[i].numpy() # (MAX_NUM * NUM_TTA *  NUM_MODELS,  4)
        img_id = image_id[i].numpy().decode()
        predString = ''
        if img_id not in mapping:
            continue # Stupid 7 files that they just added. IDK why.
        # Map Bbox to Img_id
        bbox = np.reshape(bbox, (ModelConfig.num_models, DataModule.MAX_BBOXES * DataModule.NUM_TTA, 5))
        all_bounding_boxes = []
        all_scores = []
        all_labels = [] 
        for model_box in bbox:
            # model_box: Tensor(MAX_BBOXES * NUM_TTA, 5) 
            model_box = np.reshape(model_box, (DataModule.NUM_TTA, DataModule.MAX_BBOXES, 5))
            all_model_boxes = []
            all_model_scores = []
            all_model_labels = []
            for tta_box in model_box:
                all_tta_boxes = []
                for box in tta_box:
                    if box[0] == -1 and box[1] == -1 and box[2] == -1 and box[3] == -1:
                        # Padding
                        continue
                    if box[4] == -1:
                        continue
                    x1, y1, x2, y2, obj = box.tolist()
                    all_tta_boxes.append([x1, y1, x2, y2, obj])
                if len(all_tta_boxes) == 0:
                    all_model_boxes.append(np.ones((0, 4)))
                    all_model_scores.append(np.ones((0,) ))
                    all_model_labels.append(np.ones((0, )))
                    continue
                all_tta_boxes = np.array(all_tta_boxes)
                all_model_boxes.append(all_tta_boxes[:, :4] / DataModule.IMG_SHAPE[1]) # Normalize the Bounding Boxes
                all_model_scores.append(all_tta_boxes[:, 4])
                all_model_labels.append(np.zeros_like(all_tta_boxes[:, 4]))
            # WBF on TTA Boxes 
            all_model_boxes, all_model_scores, all_model_labels = weighted_boxes_fusion(all_model_boxes, all_model_scores, all_model_labels)
            # all_model_boxes: tensor(N, 4) 
            # all_model_scores: tensor(N, )
            # all_model_labels: tensor(N, ) 
            if len(all_model_boxes) == 0:
                all_bounding_boxes.append(np.ones((0, 4)))
                all_scores.append(np.ones((0, )))
                all_labels.append(np.ones((0, )))
                continue 
            all_bounding_boxes.append(all_model_boxes)
            all_scores.append(all_model_scores)
            all_labels.append(all_model_labels)
        # Weighted box Fusion on Models 
        all_bounding_boxes, all_scores, all_labels = weighted_boxes_fusion(all_bounding_boxes, all_scores, all_labels)
        # all_bounding_boxes: Tensor(N, 4)
        # all_scores: Tensor(N, ) 
        # all_Labels: Tensor(N, ) 
        
        bbox = all_bounding_boxes * DataModule.IMG_SHAPE[0]
    
        for box in bbox:
            
            x1, y1, x2, y2 = box.tolist()
            
            x1 = round(x1)
            y1 = round(y1)
            x2 = round(x2)
            y2 = round(y2)
            
            if x1 < 0:
                x1 = 0
            elif x1 > 1024:
                x1 = 1024
                
            if y1 < 0:
                y1 = 0
            elif y1 > 1024:
                y1 = 1024
            
            if x2 < 0:
                x2 = 0
            elif x2 > 1024:
                x2 = 1024
            
            if y2 < 0:
                y2 = 0
            elif y2 > 1024:
                y2 = 1024
            
            min_val_y = min(y1, y2)
            max_val_y = max(y1, y2)
            
            min_val_x = min(x1, x2)
            max_val_x = max(x1, x2)
            
            x1 = min_val_x
            x2 = max_val_x
            
            y1 = min_val_y
            y2 = max_val_y
            
            
            # Create Bounding Box to the list 
            if predString == '':
                string_one_bbox = f"{x1} {y1} {x2} {y2}"
            else:
                string_one_bbox = f";{x1} {y1} {x2} {y2}"
            predString += string_one_bbox
        if predString == '':
            print("NONE FOUND")
            predString = 'no_box'
        mapping_GT[img_id] = (domain, predString)
    to_be_df = {'image_name': [], 'PredString': [], "domain": []}
    for row in df.iterrows():
        row = row[1]
        img_id = row.image_name
        domain, predString = mapping_GT[img_id]
        to_be_df['image_name'].append(img_id)
        to_be_df['PredString'].append(predString)
        to_be_df['domain'].append(domain)
    to_be_df['domain'] = orig_df.domain
    to_be_df['image_name'] = orig_df.image_name
    
    
    final_df = pd.DataFrame(to_be_df)
    final_df.to_csv("./submission.csv", index = False)
    return final_df
    

In [ ]:
df = post_process(predicted_bounding_boxes, predicted_image_ids)